In [1]:
import sys
sys.path.append("../")
import pandas as pd
from ortho_lib3 import *

In [2]:
class InsightfullExperiment(Experiment):
    
    def fit_inliers_auto_ensemble(self, 
                                  features, 
                                  factor=1.1, 
                                  results=None, 
                                  name=None, 
                                  thresholds = (0.0, 1.0, 0.0), 
                                  show_progress = True):
        if results is None:
            results = Results(self)
        auto_ensemble = []
        excluded = {}
        
        if show_progress is True:
            _features = log_progress(features)
        else:
            _features = features
        
        for fs in _features:
            name = fs
            _results = self.fit_inliers_ensemble([fs], factor=factor)
            
            if thresholds is not None:
                if _results.precision(name) != thresholds[1]:
                    excluded[name] = _results
                elif _results.recall(name) <= thresholds[0]:
                    excluded[name] = _results
                elif _results.accuracy(name) <= thresholds[2]:
                    excluded[name] = _results
                else:
                    results = self.fit_inliers_ensemble([fs], results=results, factor=factor)
                    auto_ensemble.append(fs)
            else:
                results = self.fit_inliers_ensemble([fs], results=results, factor=factor)
                auto_ensemble.append(fs)
        
        auto_ensemble = set(auto_ensemble)
#         features = set(features)
        if show_progress is True:
            print (f'Auto ensemble ({len(auto_ensemble)}/{len(features)}): ' + ', '.join(auto_ensemble))
        
        tps = []
        
        for feature in auto_ensemble:
            tps.append(results.get_true_positives(feature))
            
        return (self.fit_inliers_ensemble(auto_ensemble, results = results, name = 'Auto ensemble'), 
                tps)
    
    def fit_inliers_ensemble_pred(self, featureset, factor=1.1, results=None, name=None):
        if name is None:
            name = str(FeatureSet(featureset))
        y_pred = [ self.fit_inliers1(f, factor=factor) for f in featureset ]
        return np.array([ max( y ) for y in zip(*y_pred) ])

In [3]:
data_dir = '..//transformed_data/'
category = ['Category_1', 'Category_2', 'Category_3', 'Category_4']
exercise = 'AB1.txt'
extype = ['AB', 'AF', 'RF', 'EL']
pat_id = '3'
rotation = 'ZXZ'

dffs = create_dfframes(category, extype = extype, data_dir = data_dir, print_errors=False)

Patient.fs['left_shoulder'] = {'operations': ['min', 'max'], 'exercises': ['AB', 'AF']}
Patient.fs['right_shoulder'] = {'operations': ['min', 'max'], 'exercises': ['AB', 'AF']}
Patient.fs['left_arm'] = {'operations': ['min', 'max'], 'exercises': ['EL']}
Patient.fs['right_arm'] = {'operations': ['min', 'max'], 'exercises': ['EL']}
Patient.fs['diff_x_wrist'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['EL', 'AF', 'RF']}
Patient.fs['diff_y_wrist'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['EL', 'AF', 'RF']}
Patient.fs['diff_x_elbow'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['AB']}
Patient.fs['diff_y_elbow'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['AF', 'RF']}
Patient.fs['diff_y_shoulder'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['EL']}
Patient.fs['diff_z_shoulder'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['diff_z_elbow'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['diff_z_wrist'] = {'operations': ['min', 'max', 'mean'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['z_left_sh_max_el'] = {'operations': ['mean'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['z_right_sh_max_el'] = {'operations': ['mean'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['z_elbow'] = {'operations': ['max'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['z_wrist'] = {'operations': ['max'], 'exercises': ['AB', 'AF', 'RF']}
Patient.fs['lowest_max_shoulder_angle'] = {'operations': ['max'], 'exercises': ['AB', 'AF', 'RF']}

exercises = dffs_to_exercises(dffs)

dropped 130 because there is (a) missing exercise(s):  ['AB', 'AF', 'EL']
dropped 441 because there is (a) missing exercise(s):  ['AB', 'EL', 'RF']
dropped 423 because there is (a) missing exercise(s):  ['AB', 'AF', 'RF']
dropped 421 because there is (a) missing exercise(s):  ['AB', 'AF', 'RF']


In [4]:
exp = InsightfullExperiment(exercises, y_condition= lambda y: y != 'Category_1')

columns = exp.df.columns.to_numpy()

(res, tps) = exp.fit_inliers_auto_ensemble(columns)


../ortho_lib3.py:1007: RuntimeWarning:

invalid value encountered in double_scalars

../ortho_lib3.py:1007: RuntimeWarning:

invalid value encountered in double_scalars

../ortho_lib3.py:1007: RuntimeWarning:

invalid value encountered in double_scalars

../ortho_lib3.py:1007: RuntimeWarning:

invalid value encountered in double_scalars



Auto ensemble (18/84): diff_y_elbow_min_AF, diff_y_wrist_max_RF, diff_z_shoulder_mean_RF, diff_z_wrist_min_AF, diff_z_wrist_max_AB, diff_z_wrist_max_RF, diff_z_shoulder_min_AF, diff_z_elbow_min_AB, diff_y_wrist_min_EL, diff_z_elbow_max_AF, diff_z_elbow_min_RF, diff_y_elbow_min_RF, diff_z_wrist_max_AF, diff_z_wrist_mean_RF, diff_y_shoulder_mean_EL, diff_z_wrist_mean_AF, z_right_sh_max_el_mean_AB, diff_x_wrist_max_AF


In [5]:
res.report()

,features,recall,precision,accuracy
0,Auto ensemble,0.877358,1.0,0.903704
1,diff_y_elbow_min_AF,0.009434,1.0,0.222222
2,diff_y_wrist_max_RF,0.103774,1.0,0.296296
3,diff_z_shoulder_mean_RF,0.009434,1.0,0.222222
4,diff_z_wrist_min_AF,0.037736,1.0,0.244444
5,diff_z_wrist_max_AB,0.641509,1.0,0.718519
6,diff_z_wrist_max_RF,0.169811,1.0,0.348148
7,diff_z_shoulder_min_AF,0.047170,1.0,0.251852
8,diff_z_elbow_min_AB,0.018868,1.0,0.229630
9,diff_y_wrist_min_EL,0.009434,1.0,0.222222


In [6]:
counts = {}

for l in tps:
    for pat_id, f in l:
        if pat_id not in counts:
            counts[pat_id] = []
        counts[pat_id].append(f)

In [7]:
sorted_pats = {k: v for k, v in sorted(counts.items(), key=lambda item: len(item[1]))}

In [ ]:
counts_list = [len(v) for v in counts.values()]
max_count = max(counts_list)
min_count = min(counts_list)
most_obvious = [p for p, l in counts.items() if len(l) == max_count]
least_obvious = [p for p, l in counts.items() if len(l) == min_count]

print(most_obvious, least_obvious)

In [18]:
print('Showing features that selected the most obvious patients')
for mo in log_progress(most_obvious):
    print(f'features selected for {mo}:', sorted_pats[mo])

Showing features that selected the most obvious patients


features selected for 323: ['diff_z_wrist_max_AB', 'diff_z_wrist_max_RF', 'diff_z_shoulder_min_AF', 'diff_z_elbow_min_AB', 'diff_z_elbow_max_AF', 'diff_z_wrist_max_AF', 'diff_z_wrist_mean_RF', 'diff_z_wrist_mean_AF', 'diff_x_wrist_max_AF']


In [19]:
print('Showing features that selected the least obvious patients')
for mo in log_progress(least_obvious):
    print(f'features selected for {mo}:', sorted_pats[mo])

Showing features that selected the least obvious patients


features selected for 218: ['diff_z_wrist_max_AB']
features selected for 222: ['diff_z_wrist_max_AB']
features selected for 236: ['diff_z_wrist_max_AB']
features selected for 238: ['diff_z_wrist_max_AB']
features selected for 303: ['diff_z_wrist_max_AB']
features selected for 308: ['diff_z_wrist_max_AB']
features selected for 319: ['diff_z_wrist_max_AB']
features selected for 413: ['diff_z_wrist_max_AB']
features selected for 225: ['diff_y_elbow_min_RF']
features selected for 235: ['diff_y_elbow_min_RF']
features selected for 239: ['diff_y_elbow_min_RF']
features selected for 318: ['diff_y_elbow_min_RF']
features selected for 403: ['diff_y_elbow_min_RF']
features selected for 409: ['diff_y_elbow_min_RF']
features selected for 206: ['diff_x_wrist_max_AF']
features selected for 313: ['diff_x_wrist_max_AF']


In [14]:
from ortho_plot import *
dfs = []
# dfs.append(get_df(3, 23, 'RF1', with_rotations=True, data_dir='../sliced_data'))
dfs.append(get_df(1, 1, 'AB1', with_rotations=True, data_dir='../sliced_data'))
# dfs.append(get_df(3, 23, 'AF1', with_rotations=True, data_dir='../transformed_data'))

In [15]:
for df in log_progress(dfs):
    get_df_figure_3d(df, types = ['scatter', 'line'], colored_diffs = True).show()

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

